# Fine Tuning an LLM

## Load and Pre-process the dataset

In [1]:
import random

from datasets import load_dataset, load_from_disk
import spacy



def get_as_messages(data):
    headline = get_headline(data['text'])
    return {
        'messages': [
            {'role': 'user', 'content': get_prompt(headline)},
            {'role': 'assistant', 'content': headline},
        ]
    }


SEP = '#~#'
def get_headline(text):
    return text.split(SEP)[0].strip()


def get_prompt(headline):
    if random.random() < 0.1:
        return get_prompt_no_topic()
    topics = extract_topics(headline)
    if not topics:
        return get_prompt_no_topic()
    else:
        return get_prompt_topics(topics)


nlp = spacy.load('en_core_web_sm')
def extract_topics(text, max_topics=2):
    doc = nlp(text.lower()) # lower casing the text seems to result in better parsing
    noun_chunks = [chunk for chunk in doc.noun_chunks if chunk.root.pos_ != 'PRON'] # get all noun chunks, ignoring pronouns
    noun_chunks.sort(key=key, reverse=True)
    return [chunk.text.strip() for chunk in noun_chunks[:max_topics]]


def key(span):
    # Returns the importance of a span. A span is important if it contains an entity or is long.
    entity_types = ['PERSON', 'ORG', 'GPE']
    is_entity = any(token.ent_type_ in entity_types for token in span)
    length = len(span.text.strip())
    return (is_entity, length)


def get_prompt_topics(topics):
    template = random.choice([
        "Write a satirical headline about {topic}.",
        "Create an Onion-style headline related to {topic}.",
        "Give me your funniest, most satirical take on {topic}.",
        "Satirize the topic in {topic} in a news headline.",
        "What would a satirical news headline about {topic} sound like?",
    ])
    return template.format(topic=' and '.join(topics))


def get_prompt_no_topic():
    return random.choice([
        'Write a satirical headline in the style of Onion News.',
        'Generate an original satirical news headline.',
        "Give me a headline that sounds like it's from the Onion.",
        'Come up with a funny, fake news headline.',
        'Write a headline that is both absurd and oddly believable.',
    ])

filepath = 'Onion-News-Guided-Prompts'
try:
    ds = load_from_disk(filepath)
except:
    ds = load_dataset("Biddls/Onion_News")
    ds = ds.map(get_as_messages, remove_columns=['text'])
    ds.save_to_disk('Onion-News-Guided-Prompts')
ds['train'][:5]

{'messages': [[{'content': 'Give me your funniest, most satirical take on relaxed marie kondo and waist-high sewage.',
    'role': 'user'},
   {'content': 'Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage',
    'role': 'assistant'}],
  [{'content': 'Give me your funniest, most satirical take on u.s. officials and correct amount.',
    'role': 'user'},
   {'content': 'U.S. Officials Call For Correct Amount Of Violence',
    'role': 'assistant'}],
  [{'content': 'Give me your funniest, most satirical take on kamala harris and communications assistant.',
    'role': 'user'},
   {'content': 'Kamala Harris Asks Communications Assistant If She Can Take Them Out For Coffee And Pick Their Brain Sometime',
    'role': 'assistant'}],
  [{'content': 'Satirize the topic in fake nursing school diploma scheme in a news headline.',
    'role': 'user'},
   {'content': '25 Arrested In Fake Nursing School Diploma Scheme',
    'role': 'assistant'}],
  [{'content': 'Come up wit

## Load the baseline model

In [2]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

torch.random.manual_seed(0)

model_path = "Qwen/Qwen3-4B-Instruct-2507"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
new_template = '''{% for message in messages %}{{'<|im_start|>' + message['role'] + '\\n' + message['content'] + '<|im_end|>' + '\\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\\n' }}{% endif %}'''

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.chat_template = new_template

tokenizer.apply_chat_template(
    ds['train'][0]['messages'],
    tokenize=False,
)

'<|im_start|>user\nGive me your funniest, most satirical take on relaxed marie kondo and waist-high sewage.<|im_end|>\n<|im_start|>assistant\nRelaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage<|im_end|>\n'

## Wrap the model in a LoRA

In [4]:
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    bias="none",
)

prepared_model = prepare_model_for_kbit_training(model)

In [5]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir='./qwen3_lora_finetuned_v2.2',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=500,
    report_to="none",
    optim='paged_adamw_8bit',
    lr_scheduler_type='linear',
    warmup_steps=5,
    seed=42,
)

trainer = SFTTrainer(
    model=prepared_model,
    args=training_args,
    train_dataset=ds['train'],
    peft_config=peft_config,
    processing_class=tokenizer,
)

Tokenizing train dataset:   0%|          | 0/33880 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/33880 [00:00<?, ? examples/s]

In [6]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,4.593600
20,2.138100
30,1.739000
40,1.690400
50,1.626000
60,1.593800
70,1.623300
80,1.634000
90,1.581200
100,1.552900


/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be

TrainOutput(global_step=4235, training_loss=1.479649245161937, metrics={'train_runtime': 5898.1363, 'train_samples_per_second': 5.744, 'train_steps_per_second': 0.718, 'total_flos': 3.528847292110848e+16, 'train_loss': 1.479649245161937})